In [68]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from numpy import array
import torch
import gc
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split

In [69]:
data = pd.read_csv("./cpu_incre.csv")
#data = data.to_numpy()
col = list(map(str, data.columns))
#x = data[col[:-1]]
#y = data[col[-1]]

In [70]:
print(data.shape)
#print(len(data))

(117, 4585)


In [76]:
train_set = data[0:80]
valid_set = data[80:117]

#print(train_set[44,169])
print('Proportion of train_set : {:.2f}%'.format(len(train_set)/len(data)))
print('Proportion of valid_set : {:.2f}%'.format(len(valid_set)/len(data)))
print(train_set)

Proportion of train_set : 0.68%
Proportion of valid_set : 0.32%
    cadvisor_version_info  container_cpu_load_average_10s  \
0                       1                               0   
1                       1                               0   
2                       1                               0   
3                       1                               0   
4                       1                               0   
..                    ...                             ...   
75                      1                               0   
76                      1                               0   
77                      1                               0   
78                      1                               0   
79                      1                               0   

    container_cpu_load_average_10s.1  container_cpu_load_average_10s.2  \
0                                  0                                 0   
1                                  0                   

In [72]:
print(train_set.shape)

(80, 4585)


In [77]:
def split_sequence(sequence, n_steps):
    x, y = list(), list()
    for i in range(len(sequence)):
        
        end_ix = i + n_steps
        
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        x.append(seq_x)
        y.append(seq_y)
    return array(x), array(y)

n_steps = 3
train_x,train_y = split_sequence(train_set.node_cpu_seconds_total.values,n_steps)
valid_x,valid_y = split_sequence(valid_set.node_cpu_seconds_total.values,n_steps)

In [78]:
print(train_x.shape)
print(train_y.shape)
print(valid_x.shape)
print(valid_y.shape)

(77, 3)
(77,)
(34, 3)
(34,)


In [79]:
class ElecDataset(Dataset):
    def __init__(self,feature,target):
        self.feature = feature
        self.target = target
    
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self,idx):
        item = self.feature[idx]
        label = self.target[idx]
        
        return item,label

In [80]:
class CNN_ForecastNet(nn.Module):
    def __init__(self):
        super(CNN_ForecastNet,self).__init__()
        self.conv1d = nn.Conv1d(3,64,kernel_size=1)
        self.relu = nn.ReLU(inplace=True)
        self.fc1 = nn.Linear(64*2,50)
        self.fc2 = nn.Linear(50,1)
        
    def forward(self,x):
        print('######')
        x = self.conv1d(x)
        x = self.relu(x)
        x = x.view(-1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        print(x.shape)
              
        return x

In [81]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model = CNN_ForecastNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.MSELoss()

In [85]:
train = ElecDataset(train_x,train_y)
valid = ElecDataset(valid_x,valid_y)
train_loader = torch.utils.data.DataLoader(train,batch_size=2,shuffle=False)
valid_loader = torch.utils.data.DataLoader(valid,batch_size=2,shuffle=False)

In [96]:
train_losses = []
valid_losses = []
def Train():
    
    running_loss = .0
    
    model.train()
    
    for idx, (inputs,labels) in enumerate(train_loader):
        #print(labels)
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        preds = model(inputs.float())
        loss = criterion(preds,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss
        
    train_loss = running_loss/len(train_loader)
    train_losses.append(train_loss.detach().numpy())
    
    print(f'train_loss {train_loss}')
    
def Valid():
    running_loss = .0
    
    model.eval()
    
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(valid_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            preds = model(inputs.float())
            loss = criterion(preds,labels)
            running_loss += loss
            
        valid_loss = running_loss/len(valid_loader)
        valid_losses.append(valid_loss.detach().numpy())
        print(f'valid_loss {valid_loss}')

In [97]:
epochs = 200
for epoch in range(epochs):
    print('epochs {}/{}'.format(epoch+1,epochs))
    Train()
    Valid()
    gc.collect()

epochs 1/200
tensor([760787.3125, 760789.6250])
######


RuntimeError: Expected 3-dimensional input for 3-dimensional weight 64 3 1, but got 2-dimensional input of size [2, 3] instead